# Accelerating Numerical Python with Numba

### Madpy Meetup
September 12, 2019

### `$ whoami`

- James Bourbeau
- Software Engineer at Quansight
- Active in the Python data science ecosystem
- jrbourbeau on GitHub / @\_\_jrbourbeau\_\_ on Twitter

## Outline

- [TL;DR](#TL;DR)

- [Background](#Background)

    - [Python code execution](#Python-code-execution)
    
    - [Why is Python slow sometimes?](#Why-is-Python-slow-sometimes?)
    
    - [NumPy](#NumPy)

- [Numba](#Numba)

    - [Overview](#Overview)
    
    - [User interface](#User-interface)
    
    - [How Numba works](#How-Numba-works)
    
    - [When things go wrong$^*$](#When-things-go-wrong$^*$)
    
    - [Additional `jit` options](#Additional-jit-options)
    
    - [Generating custom `ufunc`s with `@vectorize`](#Generating-custom-ufuncs-with-@vectorize)
    
    - [Where and when to use Numba](#Where-and-when-to-use-Numba)

- [Numba in action! UMAP](#Numba-in-action!-UMAP)

- [Summary](#Summary)

- [References](#References)

- [Resources](#Resources)


# TL;DR

[ [Back to top](#Outline) ]

We have a `calculate()` function which performs some computation we're interested in

In [ ]:
import numpy as np

In [ ]:
def calculate(a):
    trace = 0.0
    for i in range(a.shape[0]):
        trace += np.cos(a[i, i])
    return a + trace

In [ ]:
x = np.arange(10_000).reshape(100, 100)

In [ ]:
calculate(x)

We can measure how long this calculation takes using the `%timeit` magic command

In [ ]:
time_python = %timeit -o calculate(x)

To speed things up, we use Numba's `@jit` decorator

In [ ]:
from numba import jit

In [ ]:
@jit
def calculate_numba(a):
    trace = 0
    for i in range(a.shape[0]):
        trace += np.cos(a[i, i])
    return a + trace

In [ ]:
calculate_numba(x)

How much does adding `@jit` speed things up?

In [ ]:
time_numba = %timeit -o calculate_numba(x)

In [ ]:
time_python.average / time_numba.average

🎉 ⚡ 🐍 Hooray, we got a significant performance improvement with minimal code changes!

But you may be asking yourself: what if we just used NumPy for the entire calculation?

In [ ]:
def calculate_numpy(a):
    return a + np.cos(np.diagonal(a)).sum()

In [ ]:
calculate_numpy(x)

In [ ]:
time_numpy = %timeit -o calculate_numpy(x)

In [ ]:
time_numpy.average / time_numba.average

We see that the Numba version is actually faster than using just NumPy

# Background

[ [Back to top](#Outline) ]

## Python code execution

Python is sometimes referred to as an interpreted language, meaning source code is translated to native machine instructions which are executed by the interpreter. However, this isn't the whole story. Python source code is first compiled to lower-level instructions called bytecode. These bytecode instructions are then executed by the Python interpreter. 

<center>
    <img src="images/bytecode.svg"
         align="center"
         width="45%">
</center>

We can use the `dis` module in the standard library to inspect bytecode for Python source code:

In [ ]:
def add(x, y):
    return x + y

In [ ]:
from dis import dis

dis(add)

The above bytecode instructions tell the Python interpreter to:

- The first `LOAD_FAST` instruction pushes a reference to the local variable `x` onto the top of the stack
- The second `LOAD_FAST` instruction pushes a reference to the local variable `y` onto the top of the stack
- `BINARY_ADD` pops the top two items off the stack (here `x` and `y`), adds them together, then places the result on the top of the stack
- The `RETURN_VALUE` instruction returns the value on the top of the stack

Going back to our `calculate` function, we can see what that bytecode looks like too:

In [ ]:
dis(calculate)

We can see there are (as expected) more instructions and things are more complex in this case, but it's nice to see the building blocks for how Python code is executed.

For a full list of bytcode instructions, see the [`dis` module documentation](https://docs.python.org/library/dis.html). As a side note, bytecode instructions are handled by the `switch` statement beginning on [line 1064](https://github.com/python/cpython/blob/e09359112e250268eca209355abeb17abf822486/Python/ceval.c#L1064) of `Python/ceval.c`. It's kind of fun to go through and see how things are handled.

## Why is Python slow sometimes?

Python is a dynamically typed language in which variables are really just names that point to Python objects. This lets us do things like:

In [ ]:
a = 1        # a is pointing to the integer 1
a = 'whoa'   # now a is point to the string 'whoa'

This is great for writing flexible code with low development overhead. However, Python's dynamic typing comes at the cost of the Python interpreter not knowing the type of variables (e.g. interger vs. floating-point vs. string) ahead of time.

For example, we can compare an integer in C vs. and integer in Python:

<center>
    <img src="images/c-int-vs-python-int.svg"
         align="center"
         width="65%">
</center>


Adding two integers in C looks like:

```c
/* C code */
int a = 1;
int b = 2;
int c = a + b;
```

Because C is statically typed, the compiler knows `a`, `b`, and `c` are all integers by their very definition. So the addition `a + b` can be done very efficiently.

The corresponding operation in Python looks like:


```python
# Python code
a = 1
b = 2
c = a + b
```

Looking at the [source code](https://github.com/python/cpython/blob/e09359112e250268eca209355abeb17abf822486/Python/ceval.c#L1272-L1296) for how Python executes the `BINARY_ADD` bytecode instruction, you'll see that there is a lot of type checking to ensure the addition operation is carried out properly. This dynamic type checking adds additional overhead to Python that's doesn't exist in statically-typed langues like C. This overhead isn't a big deal if we're just adding two numbers, but can become a performance bottleneck in scientific computing applications where we might be adding two numbers 10,000,000 times.

For further discussion about why Python can be slower than compiled languages, see Jake VanderPlas' great [Why Python is Slow: Looking Under the Hood](https://jakevdp.github.io/blog/2014/05/09/why-python-is-slow/) blog post.

## NumPy

NumPy is a Python library for efficient numerical computing in Python. It's a foundational piece of the open source Python data science ecosystem. Other libraries like SciPy, Pandas, Scikit-lean, matplotlib, Dask, and many more build on top of NumPy. 

NumPy consists primarily of three major components: the `ndarray` object, a paradigm for efficiently operating on arrays, and a rich library of array functions. 

#### `ndarray` object

The NumPy `ndarray` is a multidimensional, homogeneous array of fixed-size items

In [ ]:
a = np.array([[1, 2, 3, 4], [5, 6, 7, 8]], dtype=np.float64)
a

In [ ]:
print(a.shape)
print(a.dtype)
print(a.strides)

#### Universal functions

NumPy has a "universal function" (or `ufunc`) paradigm for efficiently operating on NumPy arrays (potentially of different number of dimensions). `ufunc`s operate on arrays in an element-by-element fashion, support array broadcasting, type casting, and several other nice features. 

In [ ]:
np.add

In [ ]:
a = np.array([1, 2, 3, 4])
b = np.array([5, 6, 7, 8])

np.add(a, b)

Let's compare the `np.add` `ufunc` to an equivalent straightforward operation with Python `for`-loops

In [ ]:
a = np.random.random(2_000)
b = np.random.random(2_000)

In [ ]:
%%timeit 

c = np.empty(1_000, dtype=np.int64)
for idx in range(1_000):
    c[idx] = a[idx] + b[idx]

In [ ]:
%timeit c = np.add(a, b)

`ufunc`s are so performant because they push looping code down into statically-typed, compiled code in NumPy.

#### Large collections of array functions

In addition to all the standard basic math operations (`+`, `-`, `*`, `/`), NumPy also implements lots of additional functionality:

- Linear algebra (e.g. SVD, least squares, etc.)
- Special math functions (e.g. sin, cos, exp/log, polynomials)
- Logical (boolean) operations (e.g. logical and, logical or)
- Random number generation
- etc.

In [ ]:
a = np.random.random((5, 5))
a

In [ ]:
np.sin(a)

In [ ]:
np.linalg.det(a)

**In summary: NumPy is great.** It underpins most of Python's scientific computing world. However, like everything, it has its limitations.

### NumPy's limitations

At one point or another, in particular if you're implementing custom algorithms or data processing pipelines, you'll come up against NumPy's limits:

- Not every operation you may want to use is implemented as a `ufunc`

- Looping over individual array elements is very slow, but is sometimes unavoidable

- Combining several NumPy `ufunc`s into a large expression can be both hard to read and still too slow

- NumPy (mostly) does not use the parallel execution capabilities of your computer

However, there's good news. These limitations can be addressed using Numba!

<center id="Numba">
    <img src="images/numba-blue-horizontal-rgb.svg"
         width="60%">
</center>

## Overview

[ [Back to top](#Outline) ]

Numba is an [open source](https://github.com/numba/numba) **just-in-time**, **type-specialized**, **function compiler** for accelerating **numerically-focused** Python:

- **Function compiler**: Numba compiles Python functions, not entire applications, and not parts of functions. Numba does not replace your Python interpreter, but is just another Python module that can turn a function into a (usually) faster function.

- **Type-specializing**: Numba speeds up a function by generating a specialized implementation for the specific data types you're using. Python functions are designed to operate on generic data types, which makes them very flexible, but also very slow. In practice, you'll only call a function with a small number of argument types, so Numba will generate a fast implementation for each set of input types.

- **Just-in-time (JIT)**: Numba translates functions when they are first called. This ensures the compiler knows what argument types you will be using. This also allows Numba to be used interactively in a Jupyter notebook just as easily as a traditional application.

- **Numerically-focused**: Currently, Numba is focused on numerical data types, like `int`, `float`, and `complex`. There is some limited support for string processing. To get best results with Numba, you will likely be using NumPy arrays.

Speedups range from approximately 2x (compared to simple NumPy code) to 200x (compared to pure Python). Note that Numba is not a replacement for the standard CPython interpreter (like e.g. [PyPy](https://pypy.org/)) and is not a Python-to-C/C++ translator (like e.g. [Cython](https://cython.org/)).

Numba is *not* likely to help when:

- Whole program compilation

- Critical functions have already been converted to C or optimized Cython

- Algorithms are not primarily numerical (exception: Numba does pretty well with bit manipulation. See the [Fastparquet](https://github.com/dask/fastparquet) library.)

## User interface

[ [Back to top](#Outline) ]

Numba's user interface is very straightforward, you decorate functions you want Numba to compile with the decorators provided by Numba:

- `@jit` - Compiles function on-the-fly to produce efficient machine code [[docs](https://numba.pydata.org/numba-doc/latest/user/jit.html)].
- `@vectorize` - Produces NumPy `ufunc`s (with all the `ufunc` methods supported) [[docs](https://numba.pydata.org/numba-doc/latest/user/vectorize.html#vectorize)].
- `@guvectorize` - Produces NumPy generalized `ufunc`s [[docs](https://numba.pydata.org/numba-doc/latest/user/vectorize.html#guvectorize)].
- `@stencil` - Declare a function as a kernel for a stencil like operation [[docs](https://numba.pydata.org/numba-doc/latest/user/stencil.html#numba-stencil)].
- `@jitclass` - For `jit` aware classes [[docs](https://numba.pydata.org/numba-doc/latest/user/jitclass.html#jitclass)].
- `@cfunc` - Declare a function for use as a native call back (to be called from C/C++ etc) [[docs](https://numba.pydata.org/numba-doc/latest/user/cfunc.html#cfunc)].
- `@overload` - Register your own implementation of a function for use in `nopython` mode, e.g. `@overload(scipy.special.j0)` [[docs](https://numba.pydata.org/numba-doc/latest/extending/high-level.html#high-level-extending)].

In [ ]:
@jit
def add(x, y):
    return x + y

In [ ]:
add(1, 1)    # Compiles add function for (int64, int64) inputs and executes

In [ ]:
add(3.4, 9.2)    # Compiles add function for (float64, float64) inputs and executes

## How Numba works

[ [Back to top](#Outline) ]


<center>
    <img src="images/architecture.svg"
         align="center"
         width="70%">
</center>

For more information about Numba's internals, I highly recommend checking out the [documentation on Numba's architecture](https://numba.pydata.org/numba-doc/dev/developer/architecture.html).

Numba returns a dispatcher object which, when called, will compile the underlying Python function to machine code and return the result.

In [ ]:
add

In [ ]:
callable(add)

From a user's perspective, this looks and acts just like a normal Python function

In [ ]:
add(1, 1)

The original Python function is stored in the `.py_func` attribute of the compiled function

In [ ]:
add.py_func

In [ ]:
add.py_func(1, 1)

You can use the `.inspect_types()` method to display an annotated version of the function source code for each set of input types which has been compiled

In [ ]:
add.inspect_types()

## When things go wrong$^*$

[ [Back to top](#Outline) ]

Like all of us, Numba is not perfect. It doesn't support all Python language features or all objects from third-party libraries. For example, let's try to use our `add` function for adding two Pandas DataFrames.

In [ ]:
import pandas as pd

df_1 = pd.DataFrame({'a': [0, 1, 2, 3],
                     'b': [4, 5, 6, 7]})

df_2 = pd.DataFrame({'a': [8, 9, 10, 11],
                     'b': [12, 13, 14, 15]})

In [ ]:
df_1

Within the Python interpreter we can add DataFrames:

In [ ]:
df_1 + df_2

What about using our Numba-compiled `add` function?

In [ ]:
add(df_1, df_2)

Let's look at `inspect_types` again...

In [ ]:
add.inspect_types()

In [ ]:
add.signatures

We now have a `(pyobject, pyobject)` signature! That's because Numba doesn't support Pandas DataFrames and so `add(df_1, df_2)` was compiled in "object mode".

### Compilation modes

Numba has two distinct compilation modes: `nopython` mode and `object` mode.

#### `nopython` mode

When compiling a function, Numba will try to infer all the types for every variable encountered. This is called `nopython` mode (i.e. able to replace all Python `PyObject`s with types supported by Numba). Numba will operate in `nopython` mode by default. However, if type inference fails then Numba will go back and treat all variables as `PyObject` types. This is called `object` mode.

#### `object` mode

Numba-compiled functions in `object` mode treat all values as Python `PyObject`s and use the Python C API to perform all operations on these objects. As such, functions compiled in `object` mode typically aren't much faster than normal Python. Speedups come when all types can be interred.

To prevent the falling back to `object` mode, and instead raise an error, you can use `@jit(nopython=True)`. This is used so often that there's a separate decorator `@njit` which is just an alias for `@jit(nopython=True)`.

In [ ]:
from numba import njit

NOTE: Falling back to `object` mode by default is going through a [depreciation cycle](http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit) and will be turned off in the `0.47.0` release of Numba.

### Supported features

Numba maintains a list of supported Python and NumPy features:

- Python language features: http://numba.pydata.org/numba-doc/latest/reference/pysupported.html
  
- NumPy features: http://numba.pydata.org/numba-doc/latest/reference/numpysupported.html
    

## Additional `jit` options

[ [Back to top](#Outline) ]

The `@jit` decorator takes optional keyword arguments that offer some additional compilation features:

### `cache`

Option to save compiled function machine code to a file. By default on-disk caching is disabled. This allows you to avoid repeating the Numba compilation step in future Python sessions. 

In [ ]:
@njit(cache=True)
def add_cached(x, y):
    return x + y

In [ ]:
add_cached(1, 1)

Now the next time I start up this notebook and run `add_cached`, the cached machine code will be used instead of recompiling.

### `nogil`

Option to have Numba-compiled function release the [global interpreter lock (GIL)](https://docs.python.org/glossary.html#term-global-interpreter-lock) while they are being executed. This allows you to take advantage of multi-core systems by running Numba code concurrently with other threads executing Python or Numba code. By default releasing the GIL is disabled.

It's worth noting that setting `nogil=True` does not make your code thread-safe, it just simply releases the GIL.

In [ ]:
def calculate(a):
    trace = 0
    for i in range(a.shape[0]):
        trace += np.cos(a[i, i])
    return a + trace

calculate_gil = njit(calculate)                   # Will not release the GIL
calculate_nogil = njit(nogil=True)(calculate)     # Will release the GIL

In [ ]:
a = np.random.random((5_000, 5_000))

In [ ]:
np.testing.assert_allclose(calculate_gil(a), calculate_nogil(a))

We can use the `concurrent.futures` module to launch a thread pool and execute Python function calls asynchronously

In [ ]:
import concurrent.futures

In [ ]:
%%timeit -r 5 -n 1

with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    results = list(executor.map(calculate_gil, [a] * 10))

In [ ]:
%%timeit -r 5 -n 1

with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    results = list(executor.map(calculate_nogil, [a] * 10))

Running in two threads with the GIL released avoids any GIL contention issues

### `parallel`

Option to enables automatic parallelization (and related optimizations) for operations in the function known to have parallel semantics. You can consult the Numba's [automatic parallel docs](https://numba.pydata.org/numba-doc/dev/user/parallel.html) for a list of supported operations which Numba will attempt to parallelize.

You can also also construct explicit parallel loops using Numba’s `prange` instead of `range`:

In [ ]:
from numba import prange

@njit(parallel=True)
def parallel_sum(a):
    s = 0
    # Without "parallel=True" in the jit-decorator
    # the prange statement is equivalent to range
    for i in prange(a.shape[0]):
        s += a[i]
    return s 

In [ ]:
a = np.random.random(1_000_000)

In [ ]:
parallel_sum(a)

## Generating custom `ufunc`s with `@vectorize`

[ [Back to top](#Outline) ]

Numba provides an `@vectorize` decorator ([documentation](https://numba.pydata.org/numba-doc/dev/user/vectorize.html)) for generating custom `ufunc`s from Python functions. This is great for when you need something that isn't already implemented as an efficient `ufunc` in NumPy. For example, NumPy does not have a `ufunc` for the `logit` function:

$\mathrm{logit}(x) = \log\Big(\frac{p}{1 - p}\Big)$

Let's write a Python implementation of `logit`:

In [ ]:
import math

def logit(a):
    return math.log(a / (1 - a))

This works on scalars just fine, but not NumPy arrays

In [ ]:
logit(0.2)

...let's make a `logit` a `ufunc`:

In [ ]:
from numba import vectorize

@vectorize
def logit_vec(a):
    return math.log(a / (1 - a))

In [ ]:
a = np.random.random(1_000_000)
a

In [ ]:
logit_vec(a)

🎉 ⚡ 🐍

To fully appreciate what just happened, see the [NumPy docs for creating a custom `ufunc`](https://docs.scipy.org/doc/numpy/user/c-info.ufunc-tutorial.html).

## Where and when to use Numba

[ [Back to top](#Outline) ]

### Step 0

Determine if your existing code is already sufficient. Don't optimize if you don't have to.

### Step 1

Profile your code using tools like [`cProfile`](https://docs.python.org/3/library/profile.html#module-cProfile), [`line_profiler`](https://github.com/rkern/line_profiler), or [`snakeviz`](https://jiffyclub.github.io/snakeviz/). This will measure execution times for invidiual parts of your code. 

### Step 2

From profiling your code try to identify bottlenecks that would be useful to speed up. Are they already well-encapsulated functions? If not, can they be?

### Step 3

`jit` the bottlenecks. 

# Numba in action! UMAP

[ [Back to top](#Outline) ]


I thought it would be instructive to see an example of how Numba is used out in the wild. Let's looks at Uniform Manifold Approximation and Projection, or [UMAP](https://umap-learn.readthedocs.io/en/latest/), which is a dimensionality reduction technique that can be used search for a low dimensional projection of the data that has the closest possible equivalent fuzzy topological structure.

Although it's somewhat outside the scope of this talk, if you're interested in learning more about UMAP, I highly recommend watching Leland McInnes' talk at SciPy 2018 on [YouTube](https://www.youtube.com/watch?v=nq6iPZVUxZU). It's very good.

Example from https://umap-learn.readthedocs.io/en/latest/basic_usage.html#digits-data

In [ ]:
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
digits = load_digits()

In [ ]:
fig, ax_array = plt.subplots(20, 20, figsize=(10, 10))
axes = ax_array.flatten()
for i, ax in enumerate(axes):
    ax.imshow(digits.images[i], cmap='gray_r')
plt.setp(axes, xticks=[], yticks=[], frame_on=False)
plt.tight_layout(h_pad=0.5, w_pad=0.01)
plt.show()

UMAP implements the familiar scikit-learn `.fit` / `.transform` interface that we can use to learn an embedding

In [ ]:
import umap

embedding = umap.UMAP(random_state=2).fit_transform(digits.data)

Some code to make an interactive plot using [Bokeh](https://bokeh.org/)...

In [ ]:
from numba_talk import plot_embedding
plot_embedding(embedding, digits)

Looking at the source code for UMAP, we can see Numba is used to calculate distance metrics and implement custom algorithms in [umap/distances.py](https://github.com/lmcinnes/umap/blob/74a7b3e75362fa6eb3318a292e5364c12c43df62/umap/distances.py) and [umap/umap_.py](https://github.com/lmcinnes/umap/blob/74a7b3e75362fa6eb3318a292e5364c12c43df62/umap/umap_.py)

# Summary

[ [Back to top](#Outline) ]

- Numba is an open source JIT compiler for Python functions

- Currently it's primarily concerned with numerically-focused Python

- It accelerates your existing Python functions with minimal code modifications

- Identify performance bottlenecks in your code and try decorating with `@jit`


# References

[ [Back to top](#Outline) ]

- [Numba documentation](https://numba.pydata.org/numba-doc/latest/)

- There are lots of great tutorials and talks about Numba online. In particular, I suggest checking out:

    - "Numba GPU tutorial" at PyData Amsterdam 2019 by Valentin Haenel [[GitHub](https://github.com/esc/pydata-amsterdam2019-numba)][[YouTube](https://www.youtube.com/watch?v=CQDsT81GyS8)] <-- thanks Val for letting me use some of your talk materials
    
    - "How to Accelerate an Existing Codebase with Numba" at SciPy 2019 by Siu Kwan Lam & Stanley Seibert [[YouTube](https://www.youtube.com/watch?v=-4tD8kNHdXs)]

    - "Numba: Tell those C++ bullies to get lost" SciPy 2017 tutorial by Gil Forsyth & Lorena Barba [[GitHub](https://github.com/gforsyth/numba_tutorial_scipy2017)][[YouTube](https://www.youtube.com/watch?v=1AwG0T4gaO0)]

- [Why Python is Slow: Looking Under the Hood](https://jakevdp.github.io/blog/2014/05/09/why-python-is-slow/) by Jake VanderPlas

# Resources

[ [Back to top](#Outline) ]


- GitHub: https://github.com/numba/numba

- Documentation: https://numba.pydata.org/numba-doc/latest/

- Community:

    - Gitter chat room: https://gitter.im/numba/numba
    
    - Mailing list: https://groups.google.com/a/continuum.io/d/forum/numba-users
    
    - As a note, I've been really impressed how responsive both the core developers and the community as a whole have been on the Gitter channel

# Thank you!

### Questions?